### Simple baseline

In this baseline we take data from the primary source, aggregate it by squares and compute some basic features from those squares.

We then fit a gradient boosting ensemble to predict whether it was raining in this particular square & hour.

For starters, let's take a look at our data.

In [2]:
TRAIN_PATH = "../data/train_msk.tsv"
NETATMO_PATH = "../data/train_msk_netatmo.tsv"
TEST_PATH = "../data/test_msk_features.tsv"
TEST_NETATMO_PATH = "../data/test_msk_netatmo.tsv"
CITY_PREDICTIONS_PATH = "../intermediate_data/prediction_msk.csv"

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [9]:
chunks = pd.read_csv(TRAIN_PATH, sep='\t',dtype=json.load(open("../data/train_col_dtypes.json")), chunksize=500000)

In [10]:
from tqdm import tqdm

In [12]:
def dist(x1,y1,x2,y2):
    return np.linalg.norm(x1-x2) + np.linalg.norm(y2-y1)

In [22]:
temp = pd.DataFrame()
i = np.random.rand(0,10)
j=0
train = pd.DataFrame()
temp_chunk=pd.DataFrame()
for chunk in tqdm(chunks):
        temp_chunk = chunk
        temp['city_code']=chunk['city_code']
        temp['sq_x']=chunk['sq_x']
        temp['sq_y']=chunk['sq_y']
        temp['hour_hash']=chunk['hour_hash']

        temp['sqr_dist'] = dist(chunk['sq_lat'],chunk['sq_lon'],
                                chunk['cell_lat'],chunk['cell_lon'])

        temp['usr_dist'] = dist(chunk['ulat'], chunk['ulon'], 
                                chunk['cell_lat'],chunk['cell_lon'])
        temp['cell_range'] = chunk[chunk['range']>-100]['range'].mean()

        temp_hash = str(chunk['u_hashed'])+ str(chunk['ver_hash'])+ str(chunk['device_model_hash'])
        temp['usr_hash'] = abs(hash(temp_hash)) % (10 ** 8)

        temp['sig_Strength'] = chunk['SignalStrength'] - chunk['SignalStrength'].mean()

        temp['sig'] = 0.1*chunk['LocationDirection']+0.1*chunk['LocationPrecision']+0.1* chunk['LocationSpeed']
        temp['rain'] = chunk['rain']
        temp['hours_since'] = chunk['hours_since']
        train = train.append(temp, ignore_index=True)
        


0it [00:00, ?it/s]
16it [01:37,  6.97s/it]


In [16]:
train.shape

(2500000, 11)

In [17]:
train.to_csv('my_train_msc.csv', index=False)

In [18]:
train = pd.read_csv('my_train_msc.csv').shape

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#["city_code","sq_x","sq_y","hour_hash"]
#SignalStrength

In [ ]:
# plt.figure(figsize=[12,6])
# plt.subplot(1,2,1)
# plt.scatter(train['ulon'][::100],train['ulat'][::100],alpha=0.01,marker='.')
# plt.subplot(1,2,2)
# plt.hist(train['precipitation'][::100],range=[0,5],bins=30)
# plt.yscale('log')
# plt.xlabel("precipitation strength")
# plt.show()

# train.head()

__Note:__ if you're low on memory, try this:
* Most obviously, downsample data
* Read one square at a time: read it, compute features, and only then read next square
* Entries for each cell appear as subsequent rows in the dataset, so you can just read, say, 25% of the data and process it, then go for next 25%, etc.
* Delete training data and intermediate aggregations liky `groupby` after you've done with feature engineering.

#### Working with netatmo

Customer grade meteostations are excellent sources of data on rain. Alas, they're rather scarce and we're unlikely to find stations in every square/time block. Therefore we're gonna need to quickly find ones from neighboring blocks.

For performance reasons, we'll use fast nearest neighbor lookup methods from sklearn.
Note that those are not the fastest neighbor lookup methods available, but they should be enough for the baseline.

We'll query the users that have neighboring longitude/lattitude within this hour. In this baseline we implicitly compute euclidian distance over latitude/longitude axes which has a number of problems: the distance gets larger as you move from equator to the poles. More importantly, this method does not take adjacent hours into consideration.
You are invited to improve on those points in your solution :)


In [6]:
train.shape

(5, 11)

In [23]:
from sklearn.neighbors import KDTree
def preprocess_netatmo(df):
    """organizes netatmo stations into KDTrees for each distinct time frame"""
    
    df_by_hour = df.groupby('hour_hash')
    anns = {}
    for hour,stations_group in df_by_hour:
        anns[hour] = KDTree(stations_group[["netatmo_latitude","netatmo_longitude"]].values,metric='minkowski',p=2)
    
    #convert groupby to dict to get faster queries
    df_by_hour = {group:stations_group for group,stations_group in df_by_hour}
    
    return df_by_hour,anns
        

In [24]:
netatmo_groups,netatmo_anns = preprocess_netatmo(pd.read_csv(NETATMO_PATH,na_values="None",sep='\t',dtype={'hour_hash':"uint64"}))

## Feature engineering

In this baseline, we're going to aggregate all user data from a specific square and a specific hour to predict whether it's raining in this square. We'll split data into blocks by `[sq_lon,sq_lat,sq_time]` and process such blocks independently.

<img src="https://usercontent1.hubstatic.com/12943886_f520.jpg" width=240px>


The next cell defines a function that extracts features from such blocks. Feel free to add some new features here or drop those you believe to be harmful.

Also note that this isn't the only way to process such data. See the [known unknowns](#known_unknowns) section.

In [25]:
def dist(x1,y1,x2,y2):
    return np.linalg.norm(x1-x2) + np.linalg.norm(y2-y1)

In [ ]:

def extract_features(group,netatmo_groups,netatmo_anns):
    """
    Extracts all kinds of features from a dataframe containing users in one group
    """
    features = {}

    #square features
    square = {col: group[col].iloc[0] for col in group.columns}
    
#     features['square_lat'] = square['sq_lat']
#     features['square_lon'] = square['sq_lon']
#     features['time_of_day'] = square['day_hour']

#     #signal strength
#     features['signal_mean'] = group['SignalStrength'] - group['SignalStrength'].mean(axis=0)
#     features['rain'] = group['rain']
    

#     #features for each user
#     group_by_user = group.groupby('u_hashed')
#     group_by_user.apply(lambda group: group['ulat'].var()+group['ulon'].var())
    
#     features['num_users'] = len(group_by_user)
#     features['mean_entries_per_user'] = group_by_user.apply(len).mean()
#     features['mean_user_signal_var'] = group_by_user.apply(
#         lambda user_entries: user_entries['SignalStrength'].var()).mean()
    
#     #netatmo features
#     if square['hour_hash'] in netatmo_groups:
#         local_stations,neighbors = netatmo_groups[square['hour_hash']],netatmo_anns[square['hour_hash']]
#         [distances],[neighbor_ids] = neighbors.query([(square['sq_lat'],square['sq_lon'])],k=10)

#         neighbor_stations = local_stations.iloc[neighbor_ids]

#         features['distance_to_closest_station'] = np.min(distances)
#         features['mean_distance_to_station'] = np.mean(distances)

#         for colname in ['netatmo_pressure_mbar','netatmo_temperature_c','netatmo_sum_rain_24h',
#                         'netatmo_humidity_percent',"netatmo_wind_speed_kmh","netatmo_wind_gust_speed_kmh"]:
#             col = neighbor_stations[colname].dropna()
#             if len(col)!=0:
#                 features[colname+"_mean"],features[colname+"_std"] = col.mean(),col.var()
#             else:
#                 features[colname+"_mean"],features[colname+"_std"] = np.nan,np.nan

#     return features
    

We now apply it to all the squares we have.

This may take time, more so if you use complex features, so you can try to speed stuff up by using [parallel groupby-apply](https://stackoverflow.com/a/27027632) or similar.

In [ ]:
from tqdm import tqdm

groupby = train.groupby(["city_code","sq_x","sq_y","hour_hash"])

X,y,block_ids = [],[],[]

for block_id in tqdm(groupby.groups):
    group = groupby.get_group(block_id)
    X.append(group,netatmo_groups,netatmo_anns)
    y.append(group.iloc[0]['rain'])
    block_ids.append(block_id+(group.iloc[0]["hours_since"],))

X = pd.DataFrame(X).fillna(-999.)
y = np.array(y)
block_ids = pd.DataFrame(block_ids,columns=["city_code","sq_x","sq_y","hour_hash","hours_since"])

__Note:__ If you're low on memory, it's time to either delete train & groupby or pickle X/y/block_ids and restart.

### Classifier

Once the data is processed, it's time to train some machine learning model that would predict rain given all features we gathered.

Since our features are all of different nature and unit scale (hours,decibels,degrees,etc.), it makes sense to use decision tree-based methods to for classification.


<img src="http://zdnet2.cbsistatic.com/hub/i/2017/07/18/d3f47c3e-8529-4855-a0e1-c686ee3b4007/d1113adf74bb59c3b46419a531c39c3e/orig.png" width=320>
In particular, we apply [CatBoost](https://catboost.yandex/), Yandex' recent open source gradient boosting implementation.

To make this baseline simple, we use catboost with default settings. You can certainly find a better combination of parameters. 

Here's a [guide](https://tech.yandex.com/catboost/doc/dg/concepts/parameter-tuning-docpage/) on how catboost hyperparameters work.

In [ ]:
in_train = block_ids['hours_since'] <= np.percentile(block_ids['hours_since'],85) #leave last 15% for validation

X_train,y_train = X[in_train],y[in_train]
X_val,y_val = X[~in_train],y[~in_train]
print("Training samples: %i; Validation samples: %i"%(len(X_train),len(X_val)))

In [ ]:
import xgboost 

In [ ]:
# #if you don't have catboost installed, use !pip install catboost
# from catboost import CatBoostClassifier

# model = CatBoostClassifier().fit(X_train,y_train)

In [ ]:
# from catboost import CatBoostClassifier

In [ ]:
model = xgboost.XGBClassifier(max_depth=3,n_estimators=200)
# Fit model
model.fit(X_train, y_train)
# Get predicted classes
preds_class = model.predict(X_test)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(X_test)
# Get predicted RawFormulaVal
#preds_raw = model.predict(test_data, prediction_type='RawFormulaVal')

In [ ]:
model.fit(train_data, train_labels, cat_features)

In [ ]:
model = xgboost.XGBClassifier(max_depth=3,n_estimators=200)
# Fit model
model.fit(X_train, y_train)
# Get predicted classes
preds_class = model.predict(X_test)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(X_test)
# Get predicted RawFormulaVal
#preds_raw = model.predict(test_data, prediction_type='RawFormulaVal')

### Analyzing results

Here you can see importances of all individual features, ranked from worst to best.



In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve
import matplotlib.pyplot as plt
%matplotlib inline

y_train_pred = model.predict_proba(X_train)[:,1]
print("Train ROC AUC:",roc_auc_score(y_train,y_train_pred))

fpr,tpr,_ = roc_curve(y_train, y_train_pred)
plt.plot(fpr,tpr,label='train AUC')

y_val_pred = model.predict_proba(X_val)[:,1]
print("Val ROC AUC:",roc_auc_score(y_val,y_val_pred))

fpr,tpr,_ = roc_curve(y_val, y_val_pred)
plt.plot(fpr,tpr,label='validation AUC')

plt.plot([0,1],[0,1])
plt.legend(loc='lower right')

In [ ]:
order = np.argsort(model._feature_importance)
plt.figure(figsize=[6,9])
plt.plot(np.array(model._feature_importance)[order],range(len(order)),marker='o')
plt.hlines(range(len(order)),np.zeros_like(order),np.array(model._feature_importance)[order],linestyles=':')
plt.yticks(range(X.shape[1]),X.columns[order]);
plt.tick_params(labelsize=16)
plt.xlim([0.1,max(model._feature_importance)*1.5])
plt.ylim(-1,len(order))
plt.xscale('log')

## Final model and uploading the results


Competition data contains three cities: Moscow, Saint-Petersburg and Kazan. To submit a prediction, you'll have to run this baseline three times separately for each city and concatenate the results. 

The code assumes that you ran this solution for each city (see comments below).

In [ ]:
#Train the model on full data. Copy model definition here.
model = CatBoostClassifier().fit(X,y)

In [ ]:

test = pd.read_csv(TEST_PATH, sep='\t',dtype=json.load(open("./data/test_col_dtypes.json")),)
test_groupby = test.groupby(["city_code","sq_x","sq_y","hour_hash"])
test_netatmo_groups,test_netatmo_anns = preprocess_netatmo(pd.read_csv(TEST_NETATMO_PATH,na_values="None",
                                                                       sep='\t',dtype={'hour_hash':"uint64"}))


In [ ]:
X_test,test_block_ids = [],[]
for block_id in tqdm(test_groupby.groups):
    group = test_groupby.get_group(block_id)
    X_test.append(extract_features(group,test_netatmo_groups,test_netatmo_anns))
    test_block_ids.append(block_id)
    
X_test = pd.DataFrame(X_test)
test_block_ids = pd.DataFrame(test_block_ids,columns=["city_code","sq_x","sq_y","hour_hash"])

In [ ]:
#This code saves the prediction for one city.
prediction_for_one_city = test_block_ids.copy()
prediction_for_one_city["prediction"] = model.predict_proba(X_test)[:,1]
prediction_for_one_city.to_csv(CITY_PREDICTIONS_PATH)

prediction_for_one_city.head()

#WARNING! you must run this notebook for all three regions before proceeding!
#We assume that you have prediction_msk.csv , prediction_spb.csv and prediction_kazan.csv files prepared.

In [ ]:
data = X.copy()
data["target"] = y
data.to_csv("intermediate_data/msk.csv")
X_test.to_csv("intermediate_data/msk_test.csv")

Gather all predictions and make submission file.


 ```
 
 ```

![img](https://images-na.ssl-images-amazon.com/images/I/31la29lBQxL.jpg)


 ```
 
 ```
